In [ ]:
def print_supertree_legible_fusionado(
    node,
    feature_names,
    class_names,
    numeric_features,
    scaler,  # dict con mean y std
    global_mapping,
    depth=0
):
    indent = "|   " * depth
    if node is None:
        print(f"{indent}[Nodo None]")
        return

    if getattr(node, "is_leaf", False):
        class_idx = int(np.argmax(node.labels))
        print(f"{indent}class: {class_names[class_idx]} (pred: {node.labels})")
        return

    feat_idx = node.feat
    feat_name = feature_names[feat_idx]
    intervals = node.intervals
    children = node.children

    if feat_name in numeric_features:
        idx = numeric_features.index(feat_name)
        mean = scaler["mean"][idx]
        std = scaler["std"][idx]
        # Construir los límites reales
        bounds = [-np.inf] + list(intervals)
        for i, child in enumerate(children):
            left = bounds[i]
            right = bounds[i+1]
            left_real = left * std + mean if np.isfinite(left) else -np.inf
            right_real = right * std + mean if np.isfinite(right) else np.inf

            if i == 0:
                cond = f"{feat_name} ≤ {right_real:.2f}"
            elif i == len(children)-1:
                cond = f"{feat_name} > {left_real:.2f}"
            else:
                cond = f"{feat_name} ∈ ({left_real:.2f}, {right_real:.2f}]"
            print(f"{indent}{cond}")
            print_supertree_legible_fusionado(
                child, feature_names, class_names, numeric_features, scaler, global_mapping, depth + 1
            )

    elif "=" in feat_name:
        var, val = feat_name.split("=", 1)
        var = var.strip()
        val = val.strip()
        for i, child in enumerate(children):
            cond = f'{var} == "{val}"' if i == 0 else f'{var} != "{val}"'
            print(f"{indent}{cond}")
            print_supertree_legible_fusionado(
                child, feature_names, class_names, numeric_features, scaler, global_mapping, depth + 1
            )
    else:
        print(f"{indent}{feat_name} [tipo desconocido]")
        for child in children:
            print_supertree_legible_fusionado(
                child, feature_names, class_names, numeric_features, scaler, global_mapping, depth + 1
            )
